In [1]:
from functions.load_data import *
from functions.attacks import *
from functions.models import *
from tensorflow.keras import losses, initializers
import csv

In [ ]:
# Experiment: w
iterations = 50 #aribitrary
loss_object = losses.BinaryCrossentropy()

for i in range(iterations):
    print(f'================\n\t{i}\n================')
    x_train, x_test, y_train, y_test = loadCIFARData() # need to try and alter so that reading and shuffling are 2 different methods

    reluModel = buildReLuModel(x_train, y_train)
    tropModel1 = buildTropicalModel(x_train, y_train, initializer_w = initializers.random_normal)
    tropModel2 = buildTropicalModel(x_train, y_train, initializer_w = initializers.RandomUniform(minval=-0.5, maxval=0.5))
    tropModel3 = buildTropicalModel(x_train, y_train, initializer_w = initializers.Constant(value=0))
    tropModel4 = buildTropicalModel(x_train, y_train, initializer_w = initializers.RandomNormal(mean=0, stddev=0.005))

    relu_x_test_att = attackTestSet(reluModel, x_test, y_test, loss_object, "ReLu")
    trop_x_test_att1 = attackTestSet(tropModel1, x_test, y_test, loss_object, "Tropical")
    trop_x_test_att2 = attackTestSet(tropModel2, x_test, y_test, loss_object, "Tropical")
    trop_x_test_att3 = attackTestSet(tropModel3, x_test, y_test, loss_object, "Tropical")
    trop_x_test_att4 = attackTestSet(tropModel4, x_test, y_test, loss_object, "Tropical")
    
    relu_pre_loss, relu_pre_acc = reluModel.evaluate(x_test, y_test)
    relu_post_loss, relu_post_acc = reluModel.evaluate(relu_x_test_att, y_test)

    trop_pre_loss1, trop_pre_acc1 = tropModel1.evaluate(x_test, y_test)
    trop_post_loss1, trop_post_acc1 = tropModel1.evaluate(trop_x_test_att1, y_test)

    trop_pre_loss2, trop_pre_acc2 = tropModel2.evaluate(x_test, y_test)
    trop_post_loss2, trop_post_acc2 = tropModel2.evaluate(trop_x_test_att2, y_test)

    trop_pre_loss3, trop_pre_acc3 = tropModel3.evaluate(x_test, y_test)
    trop_post_loss3, trop_post_acc3 = tropModel3.evaluate(trop_x_test_att3, y_test)

    trop_pre_loss4, trop_pre_acc4 = tropModel4.evaluate(x_test, y_test)
    trop_post_loss4, trop_post_acc4 = tropModel4.evaluate(trop_x_test_att4, y_test)

    # - write to csv - 
    data = [relu_pre_loss, relu_pre_acc, relu_post_loss, relu_post_acc, 
            trop_pre_loss1, trop_pre_acc1, trop_post_loss1, trop_post_acc1,
            trop_pre_loss2, trop_pre_acc2, trop_post_loss2, trop_post_acc2,
            trop_pre_loss3, trop_pre_acc3, trop_post_loss3, trop_post_acc3,
            trop_pre_loss4, trop_pre_acc4, trop_post_loss4, trop_post_acc4]
    with open('output1.csv', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(data)

In [5]:
x_train, x_test, y_train, y_test = loadCIFARData()
model = buildTropicalModel(x_train, 
                           y_train, 
                           num_epochs = 10,
                           first_layer_size = 100, 
                           verbose = 0, 
                           initializer_w = initializers.random_normal, 
                           clipnorm = None,
                           clipvalue = None)

loss_object = losses.BinaryCrossentropy()
x_test_attacked = attackTestSet(model, x_test, y_test, loss_object, "ReLu", attack='pgd', epsilon = 8/255, num_steps = 5, step_size = 0.01)

model.evaluate(x_test, y_test)
model.evaluate(x_test_attacked, y_test)

[1.         0.98823529 0.99215686 ... 0.25098039 0.27843137 0.28235294]
(50000, 3072)
(50000, 10)
